In [ ]:
!pip install --user --upgrade pinecone-client openai
!pip install --upgrade pinecone-client openai

In [12]:
import os
from openai import OpenAI
from pinecone import Pinecone
import sqlite3
import json
from tqdm import tqdm
import time

In [ ]:
# def alter_database():
#     conn = sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\us_company_info.db')
#     c = conn.cursor()
#     c.execute('''
#               ALTER TABLE companies
#               ADD COLUMN embedding BLOB
#               ''')
#     conn.commit()
#     conn.close()

# # Call this function once to alter the database schema
# alter_database()

In [14]:
def fetch_all_companies():
    conn = sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\merging\merged_company_deduplicated.db')
    c = conn.cursor()
    c.execute('SELECT * FROM companies')
    companies = c.fetchall()
    conn.close()
    return companies

In [19]:
from dotenv import load_dotenv

load_dotenv(dotenv_path=r"C:\Users\Admin\PycharmProjects\public-comps\.env")

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

def create_embeddings(texts):
    response = client.embeddings.create(
      model="text-embedding-3-large",
      input=texts
    )
    return [data.embedding for data in response.data]


In [16]:
def update_company_embedding(company_id, embedding):
    conn = sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\merging\merged_company_deduplicated.db')
    c = conn.cursor()
    c.execute('''
              UPDATE companies
              SET embedding = ?
              WHERE id = ?
              ''', (json.dumps(embedding), company_id))
    conn.commit()
    conn.close()

In [20]:
BATCH_SIZE = 64

def extend_database_with_embeddings():
    companies = fetch_all_companies()
    company_batches = []
    current_batch = []

    for company in companies:
        company_id = company[0]
        longBusinessSummary = company[9]  # Assuming longBusinessSummary is the 9th column
        if longBusinessSummary != 'N/A':
            current_batch.append((company_id, longBusinessSummary))
            if len(current_batch) >= BATCH_SIZE:
                company_batches.append(current_batch)
                current_batch = []

    # Don't forget to process the last batch if it has remaining companies
    if current_batch:
        company_batches.append(current_batch)

    for batch in tqdm(company_batches):
        company_ids = [company[0] for company in batch]
        texts = [company[1] for company in batch]
        embeddings = create_embeddings(texts)

        for company_id, embedding in zip(company_ids, embeddings):
            update_company_embedding(company_id, embedding)

extend_database_with_embeddings()

100%|██████████| 97/97 [03:25<00:00,  2.12s/it]
